In [1]:
import great_expectations as gx

In [2]:
print(gx.__version__)

1.4.2


In [3]:
import pandas as pd

# 1. Crear un File Data Context

Un Contexto de Datos define la ubicación de almacenamiento para los metadatos, tales como sus configuraciones para Fuentes de Datos, Suites de Expectativas, Puntos de Verificación y Documentos de Datos. También contiene sus resultados de validación y las métricas asociadas a ellos, y proporciona acceso a esos objetos en Python, junto con otras funciones de ayuda para la API GX Python.

Todos los scripts que utilizan GX Core deben comenzar con la creación de un Contexto de datos.

A continuación se indican los tipos de Contexto de datos disponibles:

* **File Data Context:** Un Contexto de Datos persistente que almacena metadatos e información de configuración como archivos YAML dentro de un sistema de archivos. Los contextos de datos de archivos permiten reutilizar conjuntos de expectativas, fuentes de datos y puntos de comprobación previamente configurados.

* **Ephemeral Data Context:** Un Contexto de Datos temporal que almacena metadatos e información de configuración en memoria. Este Contexto de Datos no persistirá más allá de la sesión actual de Python. Los Contextos de Datos efímeros son útiles cuando no tienes permisos de escritura en un sistema de archivos o si vas a dedicarte a la exploración de datos sin necesidad de guardar tus resultados. 

Puede solicitar un File Data Context específico proporcionando una ruta de carpeta con el parámetro project_root_dir. Si existe un Contexto de Datos de Fichero en la carpeta especificada, se instanciará y se devolverá. Si no se encuentra un Contexto de Datos de Fichero en la carpeta especificada, se creará un nuevo Contexto de Datos de Fichero.

In [6]:
context = gx.get_context(
    mode="file",
    project_root_dir="../data_context"
)

In [5]:
print(context)

{
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 4.0,
  "data_context_id": "1164e65a-2099-4050-9951-10f80ef8f92e",
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {},
  "plugins_directory": "plugins/",
  "stores": {
    "expectations_store": {
      "class_name": "ExpectationsStore",
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "expectations/"
      }
    },
    "validation_results_store": {
      "class_name": "ValidationResultsStore",
      "store_backend": {
 

# 2. Conectarse a un Filesystem data

Los datos del sistema de archivos consisten en datos almacenados en formatos de archivo como .csv o .parquet, y ubicados en un entorno con una jerarquía de carpetas como Amazon S3, Azure Blob Storage, Google Cloud Storage, o sistemas de archivos locales y en red. GX puede utilizar pandas o Spark para leer estos datos.

Para conectarse a los datos de su sistema de archivos, primero debe crear una fuente de datos que indique a GX dónde residen sus archivos de datos. A continuación, configure los Activos de Datos para su Fuente de Datos para indicar a GX a qué conjuntos de registros desea poder acceder desde su Fuente de Datos. Por último, definirá las Definiciones de lote que le permitirán solicitar todos los registros recuperados de un Activo de datos o dividir aún más los registros devueltos en función de una fecha especificada.

## Crear un Data Source

Las fuentes de datos indican a GX dónde se encuentran sus datos y cómo conectarse a ellos. Con los datos del sistema de archivos, esto se hace dirigiendo GX a la carpeta o ubicación en línea que contiene los archivos de datos. GX admite el acceso a datos de sistemas de archivos desde Amazon S3, Azure Blob Storage, Google Cloud Storage y sistemas de archivos locales o en red.

1. Defina los parámetros de la fuente de datos.

La siguiente información es necesaria cuando se crea una Fuente de Datos Filesystem para un directorio local o en red:

* name: Un nombre descriptivo utilizado para hacer referencia a la Fuente de Datos. Debe ser único dentro del contexto de datos.
* base_directory: La ruta a la carpeta que contiene los archivos de datos, o la carpeta raíz de la jerarquía de directorios que contiene los archivos de datos.

Si utiliza un contexto de datos de archivo, puede indicar una ruta relativa al directorio base del contexto de datos. De lo contrario, debe proporcionar la ruta absoluta a la carpeta que contiene los datos.


In [8]:
source_folder = "../data"
data_source_name = "pragmaticos_test"

2. Añada una fuente de datos Filesystem a su contexto de datos.

GX puede utilizar pandas o Spark como backend para su Filesystem Data Source. Para crear su fuente de datos, ejecute uno de los siguientes conjuntos de código:

In [9]:
data_source = context.data_sources.add_pandas_filesystem(
    name=data_source_name,
    base_directory=source_folder
)

## Crear un Data Asset

Un Activo de Datos es una colección de registros relacionados dentro de una Fuente de Datos. Estos registros pueden estar ubicados en varios archivos, pero cada Activo de Datos sólo es capaz de leer un único formato de archivo específico que se determina en el momento de su creación. Sin embargo, una Fuente de Datos puede contener múltiples Activos de Datos que cubran diferentes formatos de archivo y grupos de registros.

GX proporciona dos tipos de Activos de Datos para las Fuentes de Datos del Sistema de Archivos: Activos de Datos de Archivos y Activos de Datos de Directorios.

* **File Data Asset:**  Los File Data Assets se utilizan para recuperar datos de archivos individuales en formatos como .csv o .parquet. El formato de archivo que puede leer un Activo de datos de archivo se determina cuando se crea el Activo de datos de archivo. El archivo específico que se lee viene determinado por las Definiciones de Lote que se añaden al Activo de Datos tras su creación.

Tanto Spark como pandas Filesystem Data Sources soportan File Data Assets para todos los entornos Filesystem soportados.

* **Directory Data Asset:** Los Activos de datos de directorio leen uno o más archivos en formatos como .csv o .parquet. El formato de archivo que puede leer un Activo de datos de directorio se determina cuando se crea el Activo de datos de directorio. Los datos de los archivos correspondientes se concatenan en una única tabla que puede recuperarse en su totalidad o dividirse en función del valor de un campo de fecha y hora.

Spark Filesystem Data Sources soporta Directory Data Assets para todos los entornos Filesystem soportados. Sin embargo, pandas Filesystem Data Sources no soporta Directory Data Assets en absoluto.

In [12]:
print(data_source_name); print(data_source)

pragmaticos_test
base_directory: ..\data
id: 03d970ef-9b9b-4d0d-a905-130e3747f94c
name: pragmaticos_test
type: pandas_filesystem



In [13]:
asset_name = "pragmaticos_csv_files"

In [14]:
file_csv_asset = data_source.add_csv_asset(name=asset_name)

## Crear una Batch Definition

Una Definición de lote permite solicitar todos los registros de un Activo de datos o un subconjunto basado en el contenido de un campo de fecha y hora.

In [19]:
batch_definition_name = "pragmaticos_test.csv"
batch_definition_path = "../data/pragmaticos_test.csv"

In [20]:
batch_definition = file_csv_asset.add_batch_definition_path(
    name=batch_definition_name,
    path=batch_definition_path
)

PathNotFoundError: Provided path was not able to be resolved: ../data/pragmaticos_test.csv 

In [26]:
import great_expectations as gx
from pathlib import Path

In [27]:
# Paso 1: definir un Data Source de tipo pandas para CSVs en disco
data_source = context.data_sources.add_pandas_filesystem(
    name="csv_filesystem",
    base_directory=Path("../data")  # Ruta relativa desde notebooks/ a la carpeta data/
)

# Verifica que se creó correctamente
data_source


PandasFilesystemDatasource(type='pandas_filesystem', name='csv_filesystem', id=UUID('fe152782-bb25-4da8-87e9-a27180f62d64'), assets=[], base_directory=WindowsPath('../data'), data_context_root_directory=None)

In [28]:
# Paso 2: registrar tu archivo pragmaticos_test.csv como un File Data Asset
file_data_asset = data_source.add_csv_asset(
    name="pragmaticos_test"
)

# Muestra el objeto para confirmar
file_data_asset

CSVAsset(name='pragmaticos_test', type='csv', id=UUID('cc1a3bdc-6cf7-4b26-b571-e596d6ef7dd2'), order_by=[], batch_metadata={}, batch_definitions=[], connect_options={}, sep=None, delimiter=None, header='infer', names=None, index_col=None, usecols=None, dtype=None, engine=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=None, infer_datetime_format=None, keep_date_col=False, date_format=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal='.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, encoding_errors='strict', dialect=None, on_bad_lines='error', delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None, storage_options=None, dtype_backend=None)

In [30]:
# Paso 3 (versión Windows): usar cadena con '/' en vez de Path para evitar backslashes
batch_definition = file_data_asset.add_batch_definition_path(
    name="pragmaticos_test.csv",
    path="../data/pragmaticos_test.csv"   # ruta relativa con forward-slashes
)

# Verifica que la definición existe y apunta correctamente
batch_definition


PathNotFoundError: Provided path was not able to be resolved: ../data/pragmaticos_test.csv 